In [1]:
######    
### This function prints off the most likely predicted 
### channels for each of the cells in our dataset
#######

#######    
### Load the Model Parts
#######

import SimpleLogger as SimpleLogger

import importlib
import numpy as np

import os
import pickle

import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.utils

import model_utils

from tqdm import tqdm

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

import argparse

parent_dir = '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1'
gpu_ids = [0, 1]
batch_size = 100

parser = argparse.ArgumentParser()
parser.add_argument('--parent_dir', help='save dir')
parser.add_argument('--gpu_ids', nargs='+', type=int, default=[0], help='gpu id')
parser.add_argument('--batch_size', type=int, default=400, help='batch_size')
parser.add_argument('--overwrite', type=bool, default=False, help='overwrite existing results')
parser.add_argument('--model_dir', default='struct_model', help='Model component direcoty')
args = parser.parse_args({})

args.parent_dir = parent_dir

model_dir = args.parent_dir + os.sep + args.model_dir 

opt = pickle.load(open( '{0}/opt.pkl'.format(model_dir), "rb" ))
print(opt)

opt.gpu_ids = args.gpu_ids

torch.manual_seed(opt.myseed)
torch.cuda.manual_seed(opt.myseed)
np.random.seed(opt.myseed)

dp = model_utils.load_data_provider(opt.data_save_path, opt.imdir, opt.dataProvider)

#######    
### Load REFERENCE MODEL
#######

opt.channelInds = [0, 1, 2]
dp.opts['channelInds'] = opt.channelInds
opt.nch = len(opt.channelInds)
        
opt.nClasses = dp.get_n_classes()
opt.nRef = opt.nlatentdim

models, optimizers, _, _, opt = model_utils.load_model(opt.model_name, opt)

enc = models['enc']
dec = models['dec']

enc.train(False)
dec.train(False)

models = None
optimizers = None


print('Done loading model.')

Namespace(Diters=5, DitersAlt=100, batch_size=30, channelInds=[0, 1, 2], channels_pt1=[0, 2], channels_pt2=[0, 1, 2], critRecon='BCELoss', dataProvider='DataProvider3Dh5', data_save_path='/root//results/integrated_cell/test_aaegan/dgx_test_v4_1/data.pyt', decDRatio=1e-05, dtype='float', encDRatio=0.0001, gpu_ids=[0, 1], imdir='/root/data/ipp/ipp_17_10_25', latentDistribution='gaussian', latentSample=<function sampleGaussian at 0x7fe066b7e158>, lrDec=0.0002, lrDecD=0.0002, lrEnc=0.0002, lrEncD=0.01, model_name='aaegan3Dv6-exp', myseed=0, nClasses=11, nRef=128, nch=3, nclasses=1, ndat=19755, nepochs=150, nepochs_pt2=150, nlatentdim=128, noise=0.01, noise_std=0, optimizer='adam', saveProgressIter=1, saveStateIter=1, save_dir='/root//results/integrated_cell/test_aaegan/dgx_test_v4_1/struct_model', save_parent='/root//results/integrated_cell/test_aaegan/dgx_test_v4_1', train_module='aaegan_trainv6')


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading from /root//results/integrated_cell/test_aaegan/dgx_test_v4_1/struct_model
Done loading model.


In [2]:
import shutil
import pandas as pd

#handed off from chek, corresponds to images that are in our training set


csv_path = '/root/allen/aics/modeling/cheko/projects/datasets_rnd/2018-03-16_to_ac_ad/csvs/selected_czis.csv'

save_parent = parent_dir + os.sep + 'images_out_release_v4'
save_parent_dest = parent_dir + os.sep + 'images_out_release_v4_matches_chek'

if not os.path.exists(save_parent_dest):
    os.makedirs(save_parent_dest)

df_selected = pd.read_csv(csv_path)

print(df_selected)

# for aics_id in df_selected['aics_id']:
#     img_name = save_parent + os.sep + aics_id + '_avg.png'
#     img_name_dest = save_parent_dest + os.sep + aics_id + '_avg.png'
    
#     shutil.copyfile(img_name, img_name_dest)
    
#     img_name = save_parent + os.sep + aics_id + '.tif'
#     img_name_dest = save_parent_dest + os.sep + aics_id + '.tif'
    
#     shutil.copyfile(img_name, img_name_dest)
        

    Unnamed: 0 DeliveryDate  Index  SegmentationXyPixelSize  \
0        17077   2017_03_28      7                    0.065   
1        21021   2017_03_28     52                    0.065   
2        18166   2017_03_28      3                    0.065   
3        10327   2017_03_28     18                    0.065   
4        17854   2017_03_28     16                    0.065   
5        16840   2017_03_28     19                    0.065   
6        18510   2017_03_28      6                    0.065   
7        19804   2017_03_28     24                    0.065   
8        15723   2017_03_28     27                    0.065   
9        15358   2017_03_28     20                    0.065   
10       12205   2017_03_28      2                    0.065   
11       18034   2017_03_28     26                    0.065   
12        8756   2017_03_28     35                    0.065   
13       10225   2017_03_28      8                    0.065   
14       10113   2017_03_28     36                    0

In [3]:
import pandas as pd
from tqdm import tqdm
image_ids_csv = '/root/results/ipp/ipp_18_3_28/data/handoff-production/imageIDs.csv'

dp = model_utils.load_data_provider(opt.data_save_path, opt.imdir, opt.dataProvider)




df_image_ids = pd.read_csv(image_ids_csv, names=['aics_id', 'inputFilename', 'inputFolder'])

df_dp = dp.csv_data

df_dp = df_dp.merge(df_image_ids, how='left', on=['inputFilename'])

aics_ids = list()

for i in tqdm(range(len(df_dp))):
    aics_id = df_dp.iloc[i].aics_id
    
    if not isinstance(aics_id, str) and np.isnan(aics_id):
        print(df_dp.iloc[i].inputFilename)
        aics_id = 'nan'
    
    
    aics_ids += [aics_id + '_' + str(df_dp.iloc[i].outputThisCellIndex)]

df_dp['aics_id'] = aics_ids    


dp.csv_data = df_dp

df_dp.iloc[dp.data['test']['inds']].to_csv(parent_dir + os.sep + 'images_out_test.csv')


 61%|██████    | 13428/21967 [00:06<00:04, 2070.10it/s]

3500001157_100X_20170807_4-Scene-08-P28-E07.czi
3500001157_100X_20170807_4-Scene-08-P28-E07.czi
3500001157_100X_20170807_5-Scene-03-P33-E08.czi


100%|██████████| 21967/21967 [00:10<00:00, 2024.86it/s]


In [9]:
#handed off from chek, corresponds to mitosis images

csv_path = '/root/allen/aics/modeling/cheko/projects/datasets_rnd/2018-03-16_to_ac_ad/csvs/selected_czis_fu_net_mollys_collection.csv'

save_parent = parent_dir + os.sep + 'images_out_release_v4'
save_parent_dest = parent_dir + os.sep + 'images_out_release_v4_matches_chek_mitosis'

if not os.path.exists(save_parent_dest):
    os.makedirs(save_parent_dest)

df_dp = dp.csv_data
    
df_selected = pd.read_csv(csv_path)

for aics_id in df_selected['id']:
    img_name = save_parent + os.sep + aics_id + '_avg.png'
    
    
    if os.path.exists(img_name):
        print(img_name)
#         shutil.copyfile(img_name, img_name_dest)
        
        
        if np.sum(dp.data['train']['inds'] == np.where(df_dp['aics_id'] == aics_id)[0][0]):
            train_or_test_dir = 'train'
        else:
            train_or_test_dir = 'test'
            
        img_save_dir = save_parent_dest + os.sep + train_or_test_dir
        if not os.path.exists(img_save_dir):
            os.makedirs(img_save_dir)
            
        img_name_dest = img_save_dir + os.sep + aics_id + '_avg.png'
            
        shutil.copyfile(img_name, img_name_dest)
        

        img_name = save_parent + os.sep + aics_id + '.tif'
        img_name_dest = img_save_dir + os.sep + aics_id + '.tif'

        shutil.copyfile(img_name, img_name_dest)
        
shutil.copy(save_parent_dest, '/root/allen/aics/modeling/cheko/for_others/2018-03-28_modeling_release_mollys_collection/integrated_cell/')        

/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_286_11_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_278_1_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_303_10_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_304_8_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_352_11_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_429_17_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_445_4_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_467_4_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_495_4_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_519_1_avg.

IsADirectoryError: [Errno 21] Is a directory: '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4_matches_chek_mitosis'

In [12]:
#handed off from chek, corresponds to mitosis images
import glob


data_root = '/root/allen/aics/modeling/cheko/for_others/2018-03-23_modeling_release_v2/'

images_to_get = glob.glob(os.path.join(data_root, 'label_free_ome_tiff/test/*.tif'))

aics_ids = [os.path.split(image_path)[1][0:-4] for image_path in images_to_get]


save_parent = parent_dir + os.sep + 'images_out_release_v4'
save_parent_dest = os.path.join(data_root, 'integrated_cell')

if not os.path.exists(save_parent_dest):
    os.makedirs(save_parent_dest)

df_dp = dp.csv_data
    
for aics_id in aics_ids:
    img_name = save_parent + os.sep + aics_id + '_avg.png'
    
    
    if os.path.exists(img_name):
        print(img_name)
#         shutil.copyfile(img_name, img_name_dest)
        
        
        if np.sum(dp.data['train']['inds'] == np.where(df_dp['aics_id'] == aics_id)[0][0]):
            train_or_test_dir = 'train'
        else:
            train_or_test_dir = 'test'
            
        img_save_dir = save_parent_dest + os.sep + train_or_test_dir
        if not os.path.exists(img_save_dir):
            os.makedirs(img_save_dir)
            
        img_name_dest = img_save_dir + os.sep + aics_id + '_avg.png'
            
        shutil.copyfile(img_name, img_name_dest)
        

        img_name = save_parent + os.sep + aics_id + '.tif'
        img_name_dest = img_save_dir + os.sep + aics_id + '.tif'

        shutil.copyfile(img_name, img_name_dest)
    else:
        print('Missing: ' + img_name )
        
# shutil.copy(save_parent_dest, '/root/allen/aics/modeling/cheko/for_others/2018-03-28_modeling_release_mollys_collection/integrated_cell/')        

/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-25_7_2_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_317_7_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-10_435_7_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-10_357_3_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-17_536_2_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-11_349_9_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-16_154_12_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-10_412_19_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-25_141_8_avg.png
/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-13_520_6_avg.png


In [9]:
image_ids = [os.path.split(image_path)[1][0:-4] for image_path in images_to_get]

print(image_ids)

['AICS-25_7_2', 'AICS-12_317_7', 'AICS-10_435_7', 'AICS-10_357_3', 'AICS-17_536_2', 'AICS-11_349_9', 'AICS-16_154_12', 'AICS-10_412_19', 'AICS-25_141_8', 'AICS-13_520_6', 'AICS-25_371_6', 'AICS-16_173_1', 'AICS-10_575_1', 'AICS-13_1246_2', 'AICS-11_318_8', 'AICS-12_796_8', 'AICS-25_432_2', 'AICS-16_130_5', 'AICS-13_1001_14', 'AICS-12_374_3', 'AICS-11_581_1', 'AICS-17_677_9', 'AICS-13_1226_2', 'AICS-17_744_1']


In [20]:
print(os.path.exists(save_parent_dest))
print(os.path.exists('/root/allen/aics/modeling/cheko/for_others/2018-03-28_modeling_release_mollys_collection/integrated_cell/'))

# shutil.rmtree('/root/allen/aics/modeling/cheko/for_others/2018-03-28_modeling_release_mollys_collection/integrated_cell/')
# os.makedirs('/root/allen/aics/modeling/cheko/for_others/2018-03-28_modeling_release_mollys_collection/integrated_cell/')
shutil.copytree(save_parent_dest, '/root/allen/aics/modeling/cheko/for_others/2018-03-28_modeling_release_mollys_collection/integrated_cell/')        

True
True


FileExistsError: [Errno 17] File exists: '/root/allen/aics/modeling/cheko/for_others/2018-03-28_modeling_release_mollys_collection/integrated_cell/'

In [62]:
df_dp['aics_id']

0          AICS-10_46_1
1          AICS-10_46_2
2          AICS-10_46_3
3          AICS-10_46_4
4          AICS-10_46_6
5          AICS-10_46_7
6          AICS-10_47_1
7          AICS-10_47_2
8          AICS-10_47_4
9          AICS-10_47_5
10         AICS-10_47_6
11         AICS-10_48_3
12         AICS-10_48_4
13         AICS-10_48_5
14         AICS-10_48_7
15        AICS-10_48_10
16         AICS-10_49_1
17         AICS-10_49_3
18         AICS-10_49_4
19         AICS-10_49_5
20         AICS-10_49_6
21         AICS-10_49_8
22         AICS-10_50_2
23         AICS-10_50_4
24         AICS-10_50_5
25         AICS-10_50_6
26         AICS-10_50_8
27         AICS-10_51_1
28         AICS-10_51_3
29         AICS-10_51_4
              ...      
21937     AICS-11_967_3
21938     AICS-11_967_6
21939     AICS-11_967_8
21940     AICS-11_968_2
21941     AICS-11_968_3
21942    AICS-11_968_10
21943    AICS-11_968_12
21944    AICS-11_968_16
21945     AICS-11_969_1
21946     AICS-11_969_4
21947     AICS-1

In [18]:
for aics_id in df_dp['aics_id']:
    if 'AICS-12_881_' in aics_id:
        print(aics_id)

In [8]:
import natsort
sort = natsort.natsorted(df_dp['aics_id'])

In [17]:
sort[10100:10300]

['AICS-12_826_4',
 'AICS-12_826_7',
 'AICS-12_826_8',
 'AICS-12_827_2',
 'AICS-12_827_3',
 'AICS-12_827_6',
 'AICS-12_827_7',
 'AICS-12_827_8',
 'AICS-12_827_9',
 'AICS-12_828_1',
 'AICS-12_829_2',
 'AICS-12_829_3',
 'AICS-12_829_4',
 'AICS-12_829_5',
 'AICS-12_829_6',
 'AICS-12_830_6',
 'AICS-12_831_3',
 'AICS-12_831_4',
 'AICS-12_831_5',
 'AICS-12_831_6',
 'AICS-12_831_7',
 'AICS-12_832_2',
 'AICS-12_832_4',
 'AICS-12_833_2',
 'AICS-12_833_6',
 'AICS-12_833_9',
 'AICS-12_834_4',
 'AICS-12_834_5',
 'AICS-12_835_1',
 'AICS-12_835_3',
 'AICS-12_835_4',
 'AICS-12_835_5',
 'AICS-12_835_9',
 'AICS-12_835_10',
 'AICS-12_835_12',
 'AICS-12_836_1',
 'AICS-12_836_2',
 'AICS-12_836_4',
 'AICS-12_836_5',
 'AICS-12_837_3',
 'AICS-12_837_4',
 'AICS-12_838_1',
 'AICS-12_838_3',
 'AICS-12_839_3',
 'AICS-12_840_2',
 'AICS-12_840_4',
 'AICS-12_840_5',
 'AICS-12_841_5',
 'AICS-12_841_6',
 'AICS-12_841_7',
 'AICS-12_842_1',
 'AICS-12_842_3',
 'AICS-12_843_1',
 'AICS-12_843_4',
 'AICS-12_844_10',
 'AICS-

In [45]:
df_dp = dp.csv_data
aics_ids = df_dp.iloc[dp.data['train']['inds']]['aics_id']
    
def ismember(a, b):
    bind = {}
    for i, elt in enumerate(b):
        if elt not in bind:
            bind[elt] = i
    return [bind.get(itm, -1) for itm in a]    
    
mylist = ismember(aics_ids, df_selected['id'])

train_inds = np.where(np.array(mylist)>-1)[0]

In [43]:
import pdb
from aicsimage.io import omeTifWriter
from imgToProjection import imgtoprojection
from IPython.core.display import display
import PIL.Image
import matplotlib.pyplot as plt
import scipy.misc

from scipy.stats import norm

import pandas as pd

gpu_id = opt.gpu_ids[0]

colormap = 'hsv'
colors = plt.get_cmap(colormap)(np.linspace(0, 1, 4))

# [magenta, yellow, cyan]
# colors = [[1, 0, 1], [1, 1, 0], [0, 1, 1]]

px_size = [1,1,1]

train_or_test_split = ['test', 'train']

train_or_test_split = ['test']

img_paths_all = list()

save_parent = parent_dir + os.sep + 'images_out_release_v4'
if not os.path.exists(save_parent):
    os.makedirs(save_parent)

save_parent_selected = save_parent + '_selected' + os.sep
if not os.path.exists(save_parent_selected):
    os.makedirs(save_parent_selected)    
    
save_parent_mitosis = save_parent + '_mitosis' + os.sep
if not os.path.exists(save_parent_mitosis):
    os.makedirs(save_parent_mitosis)        
    
save_out_table = save_parent + os.sep + 'list_of_images.csv'

column_names = ['orig', 'recon'] + ['pred_' + name for name in dp.label_names] + ['train_or_test', 'orig_struct', 'img_index']

if not os.path.exists(save_parent):
    os.makedirs(save_parent)


In [38]:
import glob
import os

aics_ids = [os.path.split(path)[1][0:-4] for path in glob.glob(save_parent_selected + '/*.tif')]
aics_ids += [os.path.split(path)[1][0:-4] for path in glob.glob(save_parent + '_mitosis_selected/*.tif')]

rows = list()

for aics_id in aics_ids:
    rows += [df_dp.iloc[np.where(df_dp['aics_id'] == aics_id)[0]]]
    
df = pd.concat(rows)   

df.to_csv(save_parent + '.csv')

In [37]:
save_parent_selected

'/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4_selected/'

In [46]:
#

#######    
### Main Loop
#######



def convert_image(img):
    img = img.data[0].cpu().numpy()
    img = np.transpose(img, (3, 0, 1, 2))
    
    return img

def flatten_image(img_orig, pred_imgs):
    images_proj = list()
        
    # save flat images
#         img_orig = convert_image(img_orig)

    img = np.transpose(img_orig, (1,0,2,3))
    img = imgtoprojection(img, proj_all=True, colors = colors, global_adjust=True)
    img = np.transpose(img, (1,2,0))

    images_proj.append(img)

#     img = np.transpose(pred_imgs[1], (1,0,2,3))
#     img = imgtoprojection(img, proj_all=True, colors = colors, global_adjust=True)
#     img = np.transpose(img, (1,2,0))

#     images_proj.append(img)

    for j in range(1, len(pred_imgs)):
        img = np.transpose(pred_imgs[j], (1,0,2,3))
        img = imgtoprojection(img, proj_all=True, global_adjust=True)
        img = np.transpose(img, (1,2,0))

        images_proj.append(img)

    images_proj = np.concatenate(images_proj,1)
    
    return images_proj

n_avg = 1000
n_per_batch = 100
n_rounds = int(n_avg/n_per_batch)

train_or_test_split = ['train']

# For train or test
for train_or_test in train_or_test_split:
    ndat = dp.get_n_dat(train_or_test)
    # For each cell in the data split
    
#     dat_list = list(range(0, ndat))
    
#     #do mitosis images first
#     mitosis_inds = df_dp.iloc[dp.data[train_or_test]['inds']]['MitosisLabel'] > 0
#     not_mitosis_inds = ~mitosis_inds
    
#     dat_list = np.hstack([np.where(mitosis_inds)[0], np.where(not_mitosis_inds)[0]])
    
    dat_list = train_inds
    
    for i in tqdm(dat_list):
        
        img_index = dp.data[train_or_test]['inds'][i]
        img_class = dp.image_classes[img_index]
        img_name = os.path.basename(dp.get_image_paths([i], train_or_test)[0])[0:-3]
        
        
        pred_all_path = save_parent + os.sep + dp.csv_data['aics_id'][img_index] + '.png'
        pred_all_path_avg = save_parent + os.sep + dp.csv_data['aics_id'][img_index] + '_avg.png'
        
#         print(pred_all_path)
        
        if os.path.exists(pred_all_path_avg) and not args.overwrite:
            continue
        
        #Load the image
        img_orig = dp.get_images([i], train_or_test)
        img_orig = Variable(img_orig.cuda(gpu_id), volatile=True)
        
        #pass forward through the model
        z = enc(img_orig)
#         img_recon = dec(z)
        
        pred_imgs = list()
        img_paths = list()
        
        #print original images
        img_orig = convert_image(img_orig)
        channel_names = ['memb', img_class, 'dna']

        pred_imgs.append(img_orig)
        
        #print reconstructed images
#         img_recon = convert_image(img_recon)
#         channel_names_recon = ['memb_recon', img_class + '_recon', 'dna_recon']

#         pred_imgs.append(img_recon)
        
#         channel_names += channel_names_recon
        
        #for each structure type
        for j in range(0, dp.get_n_classes()):
            pred_class_name = dp.label_names[j]
            
            #Set the class label in log(one-hot) form
            z[0].data[0] = torch.zeros(z[0].size()).fill_(-35).cuda(gpu_id)
            z[0].data[0][j] = 0
            
            #Reference variable is set as z[1]
            
            #Set the structure variation variable to most probable
            z[-1] = Variable(torch.zeros(z[-1].size()).cuda(gpu_id), volatile=True)
            
            #generate image with these settings
            img_gen = dec(z)
            
            #convert the image and get only the GFP channel
            img_gen = convert_image(img_gen)
            img_gen = np.expand_dims(img_gen[:,1,:,:],1)
            
            channel_names.append(pred_class_name + ' pred')
            
            pred_imgs.append(img_gen)
            
#         pred_imgs_all = np.concatenate(pred_imgs,1)
        
#         # save the all-channels image (orig, recon, and predicted structures)
#         img_name = save_parent + os.sep + dp.csv_data['aics_id'][img_index] + '.tif'
        
#         pred_imgs_all *= 2**16
#         pred_imgs_all = pred_imgs_all.astype('uint16')
        
#         with omeTifWriter.OmeTifWriter(img_name, overwrite_file=True) as w:
#                 w.save(pred_imgs_all, channel_names=channel_names, pixels_physical_size=px_size)
        
#         img_flat = flatten_image(img_orig, pred_imgs)
#         scipy.misc.imsave(pred_all_path, img_flat)
        
        #######
        # 3D average
        #######      
    
        for j in range(len(z)):
            z[j] = z[j].repeat(n_per_batch,1)
    
#         pred_imgs = list()
        pdfs = list()
        
#         pred_imgs.append(img_orig)
#         pred_imgs.append(img_recon)
        
#         img_recons = list()
        
        #for each structure type
        for j in range(0, dp.get_n_classes()):
            
            pred_iter = list()
            for _ in range(n_rounds):
                pred_class_name = dp.label_names[j]

                #Set the class label in log(one-hot) form

                z[0].data = torch.zeros(z[0].size()).fill_(-35).cuda(gpu_id)
                z[0].data[:, j] = 0

                #Reference variable is set as z[1]

                #Set the structure variation variable to rando
                z[-1].data.normal_(mean=0, std=3)
                
                pdf = np.prod(norm.pdf(z[-1].data.cpu().numpy()), axis=1)
                
                pdf = (pdf / np.sum(pdf)) / n_rounds
                
                #generate image with these settings
                img_gen = dec(z)
                
                for k, p in zip(range(img_gen.shape[0]), pdf):
                    img_gen[k].data *= p
                
                
#                 img_gen = torch.sum(img_gen,dim=0)

                img_gen = torch.unsqueeze(torch.sum(img_gen, dim=0), dim=0)
                
                #convert the image and get only the GFP channel
                img_gen = convert_image(img_gen)
                img_gen = np.expand_dims(img_gen[:,1,:,:],1)

                pred_iter.append(img_gen)
                pdfs.append(pdf)
                
            channel_names.append(pred_class_name + ' avg')
            
            
#             pdb.set_trace()
            
            pred_imgs += [np.sum(np.stack(pred_iter,axis=4), axis=4)/n_rounds]
            
        pred_imgs_all = np.concatenate(pred_imgs,1)
        
        pred_imgs_all *= 2**16
        pred_imgs_all = pred_imgs_all.astype('uint16')
        
        # save the all-channels image (orig, recon, and predicted structures)
        img_name = save_parent + os.sep + dp.csv_data['aics_id'][img_index] + '.tif'
        with omeTifWriter.OmeTifWriter(img_name, overwrite_file=True) as w:
                w.save(pred_imgs_all, channel_names=channel_names, pixels_physical_size=px_size)        
        
        
        img_paths += [train_or_test, img_class, img_index]
        img_paths_all.append(img_paths)        
        
        #######
        # 2D Projection
        #######
        
        img_flat = flatten_image(img_orig, pred_imgs)
        scipy.misc.imsave(pred_all_path_avg, img_flat)

#save the list of all images
img_paths_all_df = pd.DataFrame(img_paths_all, columns=column_names);
img_paths_all_df.to_csv(save_out_table)





  0%|          | 0/19 [00:00<?, ?it/s]/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/nn/modules/container.py:72: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 19/19 [33:14<00:00, 101.42s/it]


AssertionError: 16 columns passed, passed data had 3 columns

In [81]:
suffixes = ['.tif', '_avg.png']

coords = list()
img_list = list()
id_list = list()
img_classes = list()
mitotic_labels = list()

for train_or_test in train_or_test_split:
    ndat = dp.get_n_dat(train_or_test)
    # For each cell in the data split
    for i in tqdm(range(0, ndat)):
        
        img_index = dp.data[train_or_test]['inds'][i]
        img_class = dp.image_classes[img_index]
        img_name = os.path.basename(dp.get_image_paths([i], train_or_test)[0])[0:-3]
        
        img_id = dp.csv_data['aics_id'][img_index]
        
        mitotic_label = dp.csv_data['MitosisLabel'][img_index]
        
        img_paths = [save_parent + os.sep + img_id + suffix for suffix in suffixes]
#         print(pred_all_path)
        
        if not os.path.exists(img_paths[0]):
            continue
            
            #Load the image
        img_orig = dp.get_images([i], train_or_test)
        img_orig = Variable(img_orig.cuda(gpu_id), volatile=True)
        
        #pass forward through the model
        z = enc(img_orig)
        
        coords += [z[1].data.cpu().numpy()]
        img_list += [img_paths]
        img_classes += [img_class]
        id_list += [img_id]
        mitotic_labels += [mitotic_label]
        
        
coords = np.vstack(coords)  
img_list = np.array(img_list)
mitotic_labels = np.array(mitotic_labels)
            
        

  0%|          | 0/2212 [00:00<?, ?it/s]/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/nn/modules/container.py:72: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 2212/2212 [00:35<00:00, 62.06it/s] 


In [85]:
img_list[mitotic_labels>0]

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


array([[ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_190_5.tif',
        '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_190_5_avg.png'],
       [ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-24_14_4.tif',
        '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-24_14_4_avg.png'],
       [ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-24_17_1.tif',
        '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-24_17_1_avg.png'],
       [ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-24_47_6.tif',
        '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-24_47_6_avg.png'],
       [ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-16_25_7.tif',
        '/root/resu

In [86]:
#get the cell and nuc shapes closes to the distribution dentroid

dists = np.linalg.norm(coords, axis=1)
ulabels = np.unique(img_classes)

centroid_img_list = list()

for label in ulabels:
    inds = np.where([img_class == label for img_class in img_classes])[0]

    argminind = np.argmin(dists[inds])
    img_ind = inds[argminind]
    
    centroid_img_list += [img_list[img_ind]]
    
print(centroid_img_list)  

mitosis_img_list = img_list[mitotic_labels>0]





[array([ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_52_9.tif',
       '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_52_9_avg.png'],
      dtype='<U100'), array([ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-7_99_6.tif',
       '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-7_99_6_avg.png'],
      dtype='<U100'), array([ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-16_33_6.tif',
       '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-16_33_6_avg.png'],
      dtype='<U100'), array([ '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_2_7.tif',
       '/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v4/AICS-12_2_7_avg.png'],
      dtype='<U100'), array([ '/root/results/integrated_cell/test_aaegan/

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in greater


In [88]:
import shutil


for img_list in centroid_img_list:
    for path in img_list:
        shutil.copyfile(path, save_parent_selected + os.path.split(path)[1])
        
for img_list in mitosis_img_list:
    for path in img_list:
        shutil.copyfile(path, save_parent_mitosis + os.path.split(path)[1])        

In [38]:
save_parent_selected + os.path.split(path)[1]

'/root/results/integrated_cell/test_aaegan/dgx_test_v4_1/images_out_release_v2_selected/AICS-13_223_5_avg.png'

In [55]:
mitosis_inds = df_dp.iloc[dp.data[train_or_test]['inds']]['MitosisLabel'] > 0
not_mitosis_inds = ~mitosis_inds

In [60]:
np.hstack([np.where(mitosis_inds)[0], np.where(not_mitosis_inds)[0]])

array([ 324,  436,  441, ..., 2209, 2210, 2211])

In [53]:
len(df_dp.iloc[dp.data[train_or_test]['inds']]['MitosisLabel'] > 0)

2212